In [162]:
# 모듈 Import
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from time import sleep

In [21]:
# 접속 URL
url = 'https://music.bugs.co.kr/chart'

# 응답 정보
response = requests.get(url)
print('응답 데이터 종류 : ', response.headers['Content-Type'])
print(response.text[:260])

응답 데이터 종류 :  text/html;charset=UTF-8
<!DOCTYPE html>
<html lang="ko">
<head>
	<!-- bgsw-www-t1902 -->



	<meta charset="utf-8">
	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<meta name="keywords" content="노래추천, 음악추천, 플레이리스트, essential, 에센셜, 스트리밍, 스밍, 뮤직PD, 노래듣기, 음악다운로드, 페이코" />



In [159]:
# 함수 정의
def bugs_row(tr):
    # 순위
    rank = tr.find('strong').text
    # 제목
    try:
        title = tr.find('p', {'class':'title'}).find('a').text
    except:
        title = tr.find('p', {'class':'title'}).find('span', '').text
    # 아티스트
    artist = tr.find('p', {'class':'artist'}).find('a').text
    # 앨범
    try:
        album = tr.find('a', {'class':'album'}).text 
    except:
        album = tr.find('span', {'class':'album'}).text.strip()
    # 이미지
    img_url = tr.find('img').get('src').split('?')[0]
    # 데이터 목록 생성
    row = [rank, title, artist, album]

    # 데이터 반환
    return {'info':row, 'url':img_url}

In [74]:
# BeautifulSoup 파싱
text = response.text
html = BeautifulSoup(text, 'html.parser')

# 테이블, 행 개수 확인
# 속성 : list trackList byChart
print('테이블 개수 : ', len(html.find_all('table', {'class':'list'})))
print('행 개수 : ', len(html.find('table', {'class':'list'}).find('tbody').find_all('tr')))

# 행 목록
tr_list = html.find('table', {'class':'list'}).find('tbody').find_all('tr')

# 데이터 수집
rows = []
for tr in tr_list:
    row = bugs_row(tr)
    rows.append(row)


# 데이터 프레임
chart = pd.DataFrame(rows, columns=['rank', 'title', 'artis', 'album'])
chart.set_index('rank', inplace=True)
chart

테이블 개수 :  1
행 개수 :  100


,title,artis,album
rank,,,
1,밤양갱,비비(BIBI),밤양갱
2,첫 만남은 계획대로 되지 않아,TWS (투어스),TWS 1st Mini Album ‘Sparkling Blue’
3,EASY,LE SSERAFIM (르세라핌),EASY
4,나는 아픈 건 딱 질색이니까,(여자)아이들,2
5,홀씨,아이유(IU),The Winning
...,...,...,...
96,"그대가 있는 곳, 언제 어디든",로이킴,로이킴 X 마이데몬
97,운이 좋았지,권진아,나의 모양
98,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles in December)’


In [163]:
# 변수 선언
year = 2018
month = 1

# 반복문으로 실행
for day in range(11, 13):
    # 날짜
    date = f'{year:0>4}{month:0>2}{day:0>2}'
    # 폴더 
    path = f'bugs_{date}'
    # 폴더 생성
    if not os.path.isdir(path):
        os.mkdir(path)
    
    # 요청 URL
    url = f'https://music.bugs.co.kr/chart/track/day/total?chartdate={date}'
    # URL 요청
    response = requests.get(url)

    # BeautifulSoup 파싱
    text = response.text
    html = BeautifulSoup(text, 'html.parser')

    # 행 목록
    tr_list = html.find('table', {'class':'list'}).find('tbody').find_all('tr')

    # 데이터 수집
    rows = []
    for tr in tr_list:
        # 데이터 조회
        result = bugs_row(tr)
        row = result['info']
        img_url = result['url']
        
        # def save_image(img_url, row, date, path)
        # 이미지 요청
        response2 = requests.get(img_url)
        content = response2.content
        
        # 이미지 저장
        filename = f'{path}/image_{date}_{row[0]}.jpeg'
        file = open(filename, 'wb')
        file.write(content)
        file.close()
        
        rows.append(row)

    # 데이터 프레임
    chart = pd.DataFrame(rows, columns=['rank', 'title', 'artis', 'album'])
    chart.set_index('rank', inplace=True)
    
    # 파일 저장
    filename = f'{path}/bugs_chart_{date}.csv'
    chart.to_csv(filename)
    sleep(3)

In [136]:
# 테스트 코드
date = '20180111'
url = f'https://music.bugs.co.kr/chart/track/day/total?chartdate={date}'

response = requests.get(url)
text = response.text
html = BeautifulSoup(text, 'html.parser')
tr_list = html.find('table', {'class':'list'}).find('tbody').find_all('tr')

In [142]:
# 정상
rank37 = tr_list[36]
# 비정상
rank38 = tr_list[37]

# 이미지 불러오기
print(len(rank37.find_all('img')))
url = rank37.find('img').get('src').split('?')[0]
response = requests.get(url)
content = response.content

file = open('csv/그림.jpeg', 'wb')
file.write(content)
file.close()

1
